In [ ]:
%reset

In [ ]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed, parallel_backend
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal
import sources.FermiBackgrounds as FermiBackgrounds
from astropy import units as u


%matplotlib inline

In [ ]:
grains=10_000
num_workers = 48

In [ ]:
energy_range = [1_000, 100_000] #MeV 
max_radius = 220 #kpc #This is the virial radius of the Milky Way
exposure = 2_000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut_degrees = 180 #degrees # 180 degress is full sky
lat_cut_degrees = 0 #degrees # 0 degrees is full sky
luminosity_range = np.array([1e13, 1e53]) # Minimum value set by considering source at distnace of closest approach by line of sight at 40 degrees and receiving 0.1 photon at detector side. CHANGE THIS FOR FINAL PROBLEM.
                                            # Maximum value set by that value of luminosity that suffieciently suppreses the luminoisity function at the higher end of the luminosity range.


parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
angular_cut = angular_cut_degrees*u.deg.to('rad') # radians
angular_cut_gen =  min(angular_cut*1.5, np.pi) # radians
lat_cut = lat_cut_degrees*u.deg.to('rad') # radians
lat_cut_gen = lat_cut*0.5 # radians

In [ ]:
def build_aegis_object():
    engine = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut , lat_cut, flux_cut, energy_range_gen=energy_range_gen, verbose = False)
    engine.angular_cut_gen, engine.lat_cut_gen = angular_cut_gen, lat_cut_gen
    return engine


In [ ]:
# Note: Terminology:
# flux is photon/cm^2/s
# differential flux is photon/cm^2/s/MeV

# Add Fermi background source class
data_root = '/home/users/ids29/DGRB'
fb = FermiBackgrounds.FermiBackgrounds(data_root)

def spectrum_iso_BG(energy, params): # returns spectrum dN/dA/dt/dE/d(solid angle)  for each value of energy specified in the input array 'energy' 
    #such that the area under curve equals the flux (photons/cm^2/s) over the detector's energy range

    dFdEdSr_E_interpfunc = fb.get_isotropic_background_spectrum_func() # returns an interpolation function -> dF/dE/d(solid_angle) versus E
                                
    return dFdEdSr_E_interpfunc(energy) 

# AEGIS functions for Fermi isotropic background
als_iso_BG = [spectrum_iso_BG]

In [ ]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(my_AEGIS, params): # No mock_observe applied to photon data

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-3)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    return photon_info

In [ ]:
# Instatantiate the AEGIS object for Background
obj_AEGIS = build_aegis_object()

# # Configure the AEGIS object for Isotropic Background
# obj_AEGIS.abun_lum_spec = [als_iso_BG]
# obj_AEGIS.source_class_list = ['isotropic_diffuse']

# # Run the simulator for Isotropic Background data
# photon_data_iso_BG = simulator(obj_AEGIS, torch.tensor([-1])) # Dummy params value

In [ ]:
energy_vals = np.geomspace(self.Emin_gen, self.Emax_gen, grains)
spectrum = self.abun_lum_spec[si][0](energy_vals, input_params)
solid_angle = 2*np.pi*(1-np.cos(self.angular_cut_gen))
exposure_correction = units.kpc.to('cm')**2
mean_photons = np.rint(solid_angle*np.sum(spectrum[1:]*(energy_vals[1:] - energy_vals[:-1]))*self.exposure*exposure_correction).astype('int')
num_photons = np.random.poisson(mean_photons)
if num_photons == 0:
    Es = np.array([])
else:
    Es = energy_vals[self.draw_from_pdf(energy_vals, spectrum/np.sum(spectrum), num_photons)]